In [20]:
import os
import base64
import requests
from requests import post, get
import json
from tqdm import tqdm
import pandas as pd
import time
import math
import re

In [21]:
client_credentials = [
('a235d203cfd24d37b65284e89a759ea6', '0810483fcbb0449f9bf8b4e6d2b261bc'),
('2ceaa0f0ebe0463b9b78cf6c823caade', '2ceab07ff6b240df86d5c5760fcfc891'),
('cf7714b9b1a3443095c51a479059fc98', '29d84bcb8a844bcd939abc58b801e5b4'),
('7abbe13721a64a21b66cf9401608d9d2', '85a52825264745098ef6c29d3b8aa46c'),
('4c2e3628be734c04b1b5405d3e5757e1', '09e06d8c55a6463bb65a44d232b90dcb'),
('4b0950de62b5411d96c7e9ff575f7693', '2b6c38a0ad0c4ed9a5cc4ea5f430dfe1'),
('72084958afd24bb4912bbcfb6106f1ef', '63c3fd0f87bc43a597b8d4177b04cb38'),
('2da7484f054f4ff1ac9dd13360fd7db8', '76bffc8a3eab4a53a45df2e7e70075f7'),
('ae08853620ae43edb80e8a741192a919', '3f55b7faf29a442f9675401f758909d2'),
('65eafcebb108499a9b57eaa3a0aae81f', '19ea8fa6151949229a46fe377d097752')

]



credential_index = 0

def get_token(client_id, client_secret):
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}

    response = requests.post(url, headers=headers, data=data)
    json_result = json.loads(response.content)

    if "access_token" in json_result:
        token = json_result["access_token"]
        return token
    else:
        print("Error: Failed to get access token")
        print(json_result)
        return None

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

def get_albums_for_artist(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/albums"
    headers = get_auth_header(token)
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        albums = json.loads(response.text)["items"]
        album_dict = {}
        for album in albums:
            album_id = album["id"]
            album_name = album["name"]
            album_tracks = album["total_tracks"]
            album_type = album["album_type"]
            album_release_date = album["release_date"]
            album_artists = [artist["name"] for artist in album["artists"]]
            album_images = album["images"]

            if album_name in album_dict:
                existing_tracks = album_dict[album_name]["total_tracks"]
                if album_tracks > existing_tracks:
                    album_dict[album_name] = {
                        "album_id": album_id,
                        "album_name": album_name,
                        "total_tracks": album_tracks,
                        "album_type": album_type,
                        "release_date": album_release_date,
                        "artists": album_artists,
                        "images": album_images
                    }
            else:
                album_dict[album_name] = {
                    "album_id": album_id,
                    "album_name": album_name,
                    "total_tracks": album_tracks,
                    "album_type": album_type,
                    "release_date": album_release_date,
                    "artists": album_artists,
                    "images": album_images
                }
        return album_dict
    else:
        print("Error: Failed to get albums for artist")
        print(response.content)
        return None
    
def get_tracks_for_album(token, album_id):
    url = f"https://api.spotify.com/v1/albums/{album_id}/tracks"
    headers = get_auth_header(token)
    params = {"market": "US"}
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        tracks = json.loads(response.text)["items"]
        track_list = []
        for track in tracks:
            track_name = track["name"]
            track_id = track["id"]
            track_number = track["track_number"]
            track_explicit = track["explicit"]
            track_artists = [artist["name"] for artist in track["artists"]]
            track_dict = {
                "track_name": track_name,
                "track_id": track_id,
                "track_number": track_number,
                "track_explicit": track_explicit,
                "track_artists": track_artists,
            }
            track_list.append(track_dict)
        return track_list
    else:
        print("Error: Failed to get tracks for album")
        print(response.content)
        return None

# def get_audio_features(token, track_ids):
#     headers = {"Authorization": f"Bearer {token}"}
#     url = "https://api.spotify.com/v1/audio-features"
    
#     id_chunks = [track_ids[i:i+100] for i in range(0, len(track_ids), 100)]
    
#     dfs = []
#     for chunk in id_chunks:
#         params = {"ids": ",".join(chunk)}
#         response = requests.get(url, headers=headers, params=params)
#         data = response.json()["audio_features"]
#         features = pd.DataFrame(data)
#         features = features.drop(["type", "uri", "track_href", "analysis_url"], axis=1)
#         dfs.append(features)
    
#     features = pd.concat(dfs, ignore_index=True)
#     return features

def get_audio_features(token, track_ids):
    audio_features = []

    for i in range(0, len(track_ids), 50):
        chunk = track_ids[i:i + 50]
        ids = ','.join(chunk)
        url = f"https://api.spotify.com/v1/audio-features?ids={ids}"
        headers = {"Authorization": f"Bearer {token}"}
        r = requests.get(url, headers=headers)
        if r.status_code != 200:
            print(f"Error getting audio features for tracks: {r.status_code}")
            return None

        try:
            features = r.json()["audio_features"]
        except KeyError:
            features = []

        audio_features.extend(features)

    audio_features_filtered = []
    for feature in audio_features:
        if feature is not None:
            audio_features_filtered.append(feature)

    return pd.DataFrame(audio_features_filtered)



def load_checkpoint():
    if os.path.exists('checkpoint_7.json'):
        with open('checkpoint_7.json', 'r') as f:
            return json.load(f)
    else:
        return {"artist_id": "", "progress": 0}

def save_checkpoint(artist_id, progress):
    with open('checkpoint_7.json', 'w') as f:
        json.dump({"artist_id": artist_id, "progress": progress}, f)

def load_failed_requests():
    if os.path.exists('failed_requests_7.parquet'):
        return pd.read_parquet('failed_requests_7.parquet')
    else:
        return pd.DataFrame(columns=['Spotify ID'])

def save_failed_requests(df):
    df.to_parquet('failed_requests_7.parquet')

def refresh_token(token_start_time):
    global credential_index
    current_time = time.time()
    elapsed_time = current_time - token_start_time
    if elapsed_time >= 1800:
        credential_index = (credential_index + 1) % len(client_credentials)
        client_id, client_secret = client_credentials[credential_index]
        return get_token(client_id, client_secret), current_time
    return None, token_start_time

def sanitize_filename(filename):
    return re.sub(r'[\\/:"*?<>|]+', '_', filename)



In [22]:
with open('artist_ids_sub_dict_7.json', 'r') as f:
    artist_ids = json.load(f)
len(artist_ids)

1197

In [23]:
token = get_token(client_credentials[0][0], client_credentials[0][1])
token_start_time = time.time()

checkpoint = load_checkpoint()
failed_requests = load_failed_requests()

start_from = checkpoint["artist_id"] if checkpoint["artist_id"] else list(artist_ids.keys())[0]
start_index = list(artist_ids.keys()).index(start_from)


iteration_counter = 0
sleep_interval = 50

for artist_id, artist_name in list(artist_ids.items())[start_index:]:
    sanitized_artist_name = sanitize_filename(artist_name)
    parquet_file_path = f'results/{sanitized_artist_name}_tracks.parquet'
    if os.path.exists(parquet_file_path):
        print(f"Parquet file already exists for {artist_name}, skipping...")
        continue
    try:
        refreshed_token, new_token_start_time = refresh_token(token_start_time)
        if refreshed_token:
            token = refreshed_token
            token_start_time = new_token_start_time

        albums = get_albums_for_artist(token, artist_id)

        if albums is None:
            failed_requests = failed_requests.append({"Spotify ID": artist_id}, ignore_index=True)
            save_failed_requests(failed_requests)
            continue

        album_df = pd.DataFrame.from_dict(albums, orient="index")
        album_df.index.name = "album_name"

        all_tracks_df = pd.DataFrame()
        for album_id in tqdm(album_df['album_id'], desc=f"Processing {artist_name}"):
            time.sleep(1)
            tracks = get_tracks_for_album(token, album_id)
            if tracks is None:
                failed_requests = failed_requests.append({"Spotify ID": artist_id}, ignore_index=True)
                save_failed_requests(failed_requests)
                continue

            tracks = pd.DataFrame(tracks)
            tracks['album_id'] = album_id
            all_tracks_df = pd.concat([all_tracks_df, tracks], axis=0)

        af_df = get_audio_features(token, all_tracks_df['track_id'].tolist())
        af_df = af_df.rename(columns={'id': 'track_id'})
        all_aftracks_df = pd.merge(all_tracks_df, af_df, on='track_id')
        album_all_aftracks_df = pd.merge(album_df, all_aftracks_df, on='album_id')
        sanitized_artist_name = sanitize_filename(artist_name)
        album_all_aftracks_df.to_parquet(f'results/{sanitized_artist_name}_tracks.parquet')

        save_checkpoint(artist_id, artist_name)
        iteration_counter += 1
        if iteration_counter % sleep_interval == 0:
            time.sleep(10)
            
    except Exception as e:
        print(f"Error processing {artist_name}: {e}")
        failed_requests = failed_requests.append({"Spotify ID": artist_id}, ignore_index=True)
        save_failed_requests(failed_requests)
        continue

print("Completed processing all artists.")


Parquet file already exists for The Gutter Twins, skipping...
Parquet file already exists for The Gun Club, skipping...
Parquet file already exists for The Greenhornes, skipping...
Parquet file already exists for The Graham Bond Organisation, skipping...
Parquet file already exists for The Good, the Bad & the Queen, skipping...
Parquet file already exists for The Go-Go's, skipping...
Parquet file already exists for The Go-Betweens, skipping...
Parquet file already exists for The Glitter Band, skipping...
Parquet file already exists for The Georgia Satellites, skipping...
Parquet file already exists for The Gaylads, skipping...
Parquet file already exists for The Free Design, skipping...
Parquet file already exists for The Four Tunes, skipping...
Parquet file already exists for The Foundations, skipping...
Parquet file already exists for The Fleshtones, skipping...
Parquet file already exists for The Fixx, skipping...
Parquet file already exists for The Five Stairsteps, skipping...
Parq

In [24]:
# max_retries = 3
# retry_counter = 0

# while not failed_requests.empty and retry_counter < max_retries:
#     print(f"Retrying failed requests (attempt {retry_counter + 1}/{max_retries})...")
#     retry_failed_requests = pd.DataFrame(columns=["Spotify ID"])
#     for _, row in failed_requests.iterrows():
#         artist_id = row["Spotify ID"]
#         artist_name = artist_ids[artist_id]
#         sanitized_artist_name = sanitize_filename(artist_name)
#         parquet_file_path = f'{sanitized_artist_name}_tracks.parquet'

#         if os.path.exists(parquet_file_path):
#             print(f"Parquet file already exists for {artist_name}, skipping...")
#             continue
#         try:
#             refreshed_token, new_token_start_time = refresh_token(token_start_time)
#             if refreshed_token:
#                 token = refreshed_token
#                 token_start_time = new_token_start_time
#             albums = get_albums_for_artist(token, artist_id)

#             if albums is None:
#                 failed_requests = failed_requests.append({"Spotify ID": artist_id}, ignore_index=True)
#                 save_failed_requests(failed_requests)
#                 continue

#             album_df = pd.DataFrame.from_dict(albums, orient="index")
#             album_df.index.name = "album_name"

#             all_tracks_df = pd.DataFrame()
#             for album_id in tqdm(album_df['album_id'], desc=f"Processing {artist_name}"):
#                 time.sleep(1)
#                 try:
#                     tracks = get_tracks_for_album(token, album_id)

#                     if tracks is None:
#                         failed_requests = failed_requests.append({"Spotify ID": artist_id}, ignore_index=True)
#                         save_failed_requests(failed_requests)
#                         continue

#                     tracks = pd.DataFrame(tracks)
#                     tracks['album_id'] = album_id
#                     all_tracks_df = pd.concat([all_tracks_df, tracks], axis=0)
#                 except Exception as e:
#                     print(f"Warning: Error fetching tracks for {artist_name}: {e}")

#             try:
#                 af_df = get_audio_features(token, all_tracks_df['track_id'].tolist())
#                 af_df = af_df.rename(columns={'id': 'track_id'})
#                 all_aftracks_df = pd.merge(all_tracks_df, af_df, on='track_id')
#                 album_all_aftracks_df = pd.merge(album_df, all_aftracks_df, on='album_id')
#                 sanitized_artist_name = sanitize_filename(artist_name)
#                 album_all_aftracks_df.to_parquet(f'{sanitized_artist_name}_tracks.parquet')

#                 save_checkpoint(artist_id, artist_name)
#             except Exception as e:
#                 print(f"Warning: Error fetching audio features for {artist_name}: {e}")

#             iteration_counter += 1
#             if iteration_counter % sleep_interval == 0:
#                 time.sleep(1)

#         except Exception as e:
#             print(f"Error processing {artist_name}: {e}")
#             retry_failed_requests = retry_failed_requests.append({"Spotify ID": artist_id}, ignore_index=True)

#     failed_requests = retry_failed_requests
#     retry_counter += 1

# if not failed_requests.empty:
#     print("Some requests still failed after all retries:")
#     print(failed_requests)
# else:
#     print("All failed requests have been successfully processed.")
